In [1]:
using DataFrames, Feather

In [2]:
@enum FailureType begin
    # General ones
    Unknown
    InferredFailure
    PrintingChange
    TestAmbiguities
    NewAmbiguity
    ApproxError
    DocTest
    NewExceptionType
    DownloadError
    BadBoy
    # Specific
    IRToolsParams
    ParameterUpperBound
    ModuleIR
    BoundsErrorCompiler
    Cassette
    OldMbedTLSSyntax
    GPUErrors
end

In [3]:
if !isdir("data")
    run(`tar -xvf data.tar.gz`)
end
primary  = Feather.read("data/primary.feather");
against = Feather.read("data/against.feather");

In [4]:
package_results = join(primary, against,
     on=:uuid, kind=:left, makeunique=true, indicator=:source);

In [5]:
changed_tests = filter(test->test.source == "both" &&
                 test.status != test.status_1, package_results)
fails = changed_tests[changed_tests[!, :status] .== ":fail", :];
sort!(fails , :name);
insert!(fails, size(fails, 2), fill(Unknown, size(fails, 1)), :why);

In [6]:
query(fails, s) = filter(row -> occursin(s, row[:log]), fails)    

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails))

total_unknown (generic function with 1 method)

In [23]:
# Generic ones
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "Expression: all_doctests()", DocTest)
update_reason!(fails, "does not match inferred return type", InferredFailure);
update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, "is ambiguous. Candidates:", NewAmbiguity)
update_reason!(fails, "Evaluated: \"", PrintingChange)
update_reason!(fails, "Log Test Failed at", PrintingChange)
update_reason!(fails, "Expression: occursin(r\"", PrintingChange)
update_reason!(fails, "Expression: startswith(", PrintingChange)


update_reason!(fails, "      Thrown: ", NewExceptionType)
update_reason!(fails, "The requested URL returned error", DownloadError)
update_reason!(fails, "gzip: stdin: not in gzip format", DownloadError)

# Specific ones for this release
update_reason!(fails, "Params not defined", IRToolsParams);
update_reason!(fails, "parameter_upper_bound", ParameterUpperBound);
update_reason!(fails, "Module IR does not contain specified entry function", ModuleIR);
update_reason!(fails, "Internal error: encountered unexpected error in runtime:\nBoundsError", BoundsErrorCompiler
);
update_reason!(fails, "@ Cassette", Cassette)
update_reason!(fails, "/home/pkgeval/.julia/packages/MbedTLS/", OldMbedTLSSyntax)
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "unsupported or misplaced expression \"return\" in function", GPUErrors)

# Baddies

# https://github.com/JuliaDiffusionBayes/DiffusionDefinition.jl/issues/11
update_reason!(fails, "DD.remove_curly(Vector{Float64}) == Array", BadBoy)
update_reason!(fails, "Unknown observable eltype", BadBoy)
update_reason!(fails, "ArgumentError: Invalid type: typename(SwingEq)", BadBoy)
update_reason!(fails, "/home/pkgeval/.julia/packages/Tricks/Omjx9/test/runtests.jl:89", BadBoy)
update_reason!(fails, "Judycon ~/.julia/packages/Judycon/Zc2IV/src/Judycon.jl:7", BadBoy)
update_reason!(fails, ": Error During Test at /home/pkgeval/.julia/packages/LOLTools/K3Wzy/test/loltools/championmasteryv4.jl:1", BadBoy)
update_reason!(fails, "/home/pkgeval/.julia/packages/Libtask/Zo6uM/test/ctask.jl:23", BadBoy)
update_reason!(fails, "~/.julia/packages/MonthlyDates/vCogo/src/MonthlyDates.jl:72", BadBoy)
update_reason!(fails, "type Method has no field ambig", BadBoy)
update_reason!(fails, "oadError: UndefVarError: LAST_SHOWN_LINE_INFOS not definedfatal: ", BadBoy)
update_reason!(fails, "/home/pkgeval/.julia/packages/CQLdriver/QjN09/deps/build.jl:1", BadBoy)
update_reason!(fails, "Cannot `convert` an object of type ForneyLab.Variable to an object of type Symbol", BadBoy)
update_reason!(fails, "est Failed at /home/pkgeval/.julia/packages/XPORTA/QaHQb/test/integration/filesystem.jl:41", BadBoy)
update_reason!(fails, "During Test at /home/pkgeval/.julia/packages/AlignedBinaryFormat/Nlf6L/test/runtests.jl:59", BadBoy)
update_reason!(fails, "ApproximateComputations/fS6jW/src/ASTReplacementApproximation.jl:121", BadBoy)


nothing

In [24]:
total_unknown(fails)

44

In [26]:
filter(x -> x.why == Unknown, fails).name[1:20]

20-element Array{String,1}:
 "BlobTracking"
 "CORBITS"
 "Cassette"
 "CodeTracking"
 "CombinedParsers"
 "CopositiveAnalyticCenter"
 "CorticalSpectralTemporalResponses"
 "Cthulhu"
 "Darknet"
 "DataAPI"
 "DataFrames"
 "DevIL"
 "DiffEqBiological"
 "DimensionalData"
 "Fermi"
 "GeoGreensFunctions"
 "Graphene"
 "GslibIO"
 "IncompleteLU"
 "InterProcessCommunication"

In [21]:
count(x -> x.why == ModuleIR, eachrow(query(fails, "FunctionWrappers")))

11

In [11]:
z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    push!(z, (n, i))
end
sort!(z; rev=true)

17-element Array{Any,1}:
 (58, ParameterUpperBound)
 (48, Unknown)
 (40, PrintingChange)
 (37, ModuleIR)
 (32, IRToolsParams)
 (15, DocTest)
 (13, BadBoy)
 (12, Cassette)
 (7, BoundsErrorCompiler)
 (7, NewAmbiguity)
 (5, GPUErrors)
 (5, OldMbedTLSSyntax)
 (4, NewExceptionType)
 (3, DownloadError)
 (3, InferredFailure)
 (2, ApproxError)
 (2, TestAmbiguities)